实现图像浮雕，卷积核大小为3,5

In [6]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

# 初始化方向和大小
direction = 1
filter_size = 2
# 读取图像
src_img = cv2.imread('./input/apple.jpg')

def update_img(val):
    global direction, filter_size, src_img
    direction = cv2.getTrackbarPos('Direction', 'image')
    filter_size = cv2.getTrackbarPos('Filter Size', 'image')
    
    # 定义不同 size 下的卷积核方向
    filter_3 = [np.array([[0, -1, -1], [1, 0, -1], [1, 1, 0]]), 
                np.array([[-1, -1, 0], [-1, 0, 1], [0, 1, 1]]), 
                np.array([[1, 1, 0], [1, 0, -1], [0, -1, -1]]), 
                np.array([[-1, -1, 0], [-1, 0, 1], [0, 1, 1]])]
    
    filter_5 = [np.array([[0,-1,-1,-1,-1],[1,0,-1,-1,-1],[1,1,0,-1,-1],[1,1,1,0,-1],[1,1,1,1,0]]),
                np.array([[-1,-1,-1,-1,0],[-1,-1,-1,0,1],[-1,-1,0,1,1],[-1,0,1,1,1],[0,1,1,1,1]]),
                np.array([[1,1,1,1,0],[1,1,1,0,-1],[1,1,0,-1,-1],[1,0,-1,-1,-1],[0,-1,-1,-1,-1]]),
                np.array([[-1,-1,-1,-1,0],[-1,-1,-1,0,1],[-1,-1,0,1,1],[-1,0,1,1,1],[0,1,1,1,1]])]
    
    # 提取卷积核
    if filter_size == 3:
        filter_result = filter_3[direction]
    elif filter_size == 5:
        filter_result = filter_5[direction]
    else:
        filter_result = np.array([[1]])  # 默认为 1x1 的卷积核
    
    # 处理图像
    new_img = img_filter(src_img, filter_result)
    
    # 将单通道图像转换成多通道图像，以便拼接
    new_img = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
    compare = np.hstack((src_img, new_img))
    
    # 显示更新后的图像
    cv2.imshow('image', compare)

def img_filter(src_img, filter):
    gray_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2GRAY)
    new_img = cv2.filter2D(gray_img, -1, filter)+128
    return new_img

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.createTrackbar('Filter Size', 'image', filter_size, 5, update_img)
cv2.createTrackbar('Direction', 'image', direction, 3, update_img)

cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


用多种方法生成草图

In [2]:
src_img1 = cv2.imread('./input/apple.jpg')

def generate_sketch(val):
    global src_img1
    if val == 0:
        gray_img = cv2.cvtColor(src_img1, cv2.COLOR_BGR2GRAY)
        anti_gray_img = 255-gray_img
        blur_img = cv2.GaussianBlur(anti_gray_img,(49,49),0)
        inverted_blur_img = 255-blur_img
        dst_sketch = cv2.divide(gray_img, inverted_blur_img, scale=256.0)
    elif val == 1:
        dst_sketch,dst_color = cv2.pencilSketch(src_img1, sigma_s=60, sigma_r=0.07, shade_factor=0.05)
    else:
        gray_img = cv2.cvtColor(src_img1, cv2.COLOR_BGR2GRAY)
        gray_img = cv2.medianBlur(gray_img,7)
        edges = cv2.Laplacian(gray_img, cv2.CV_8U ,ksize=5)
        ret, mask = cv2.threshold(edges, 100, 255, cv2.THRESH_BINARY_INV)
        dst_sketch = mask
    sketch_img = cv2.cvtColor(dst_sketch, cv2.COLOR_GRAY2BGR)
    result = np.hstack((src_img1, sketch_img))
    cv2.imshow('image', result)

cv2.namedWindow('image', cv2.WINDOW_NORMAL)
cv2.createTrackbar('Direction', 'image', 1, 2, generate_sketch)
cv2.waitKey(0)
cv2.destroyAllWindows()
    